In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path
import glob 
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [2]:
def extract_features_from_images(img_paths, labels, save_dir_path=None):
    df = pd.DataFrame(columns=labels)
    skipped_img_count = 0

    hands =  mp_hands.Hands(static_image_mode=True, 
                            max_num_hands=1, 
                            min_detection_confidence=0.5)

    for idx, img_path in enumerate(img_paths):
        multi_hand_landmarks = extract_multihand_landmarks(img_path, hands)

        if not multi_hand_landmarks:
            skipped_img_count += 1
            continue
        
        first_hand_landmarks = multi_hand_landmarks[0]
        df_series = fill_df_series(first_hand_landmarks, df.columns)
        df = df.append(df_series, ignore_index=True)

        if save_dir_path is not None:
            save_annotated_image(img_path, save_dir_path, first_hand_landmarks)

    print(f"skipped images count f{skipped_img_count}")    
    return df

def fill_df_series(hand_landmarks, labels):
    df_row = []
    for finger_part in hand_landmarks.landmark:
        df_row.extend([finger_part.x, finger_part.y])
    return pd.Series(df_row, index=labels)

def extract_multihand_landmarks(hands, image_path):
    image = cv2.flip(cv2.imread(image_path), 1)
    return hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

def save_annotated_image(img_path, save_dir_path, hand_landmarks):
    annotated_image = cv2.imread(img_path)
    save_dir_path.mkdir(parents=True, exist_ok=True)
    mp_drawing.draw_landmarks(
      annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    cv2.imwrite(str(save_dir_path / Path(img_path).name), 
                annotated_image)

In [3]:
feat_labels = [
    'WRIST_X', 'WRIST_Y', 
    'THUMB_CMC_X', 'THUMB_CMC_Y',
    'THUMB_MCP_X', 'THUMB_MCP_X',
    'THUMB_IP_X', 'THUMB_IP_X',
    'THUMB_TIP_X', 'THUMB_TIP_X',
    'INDEX_FINGER_MCP_X', 'INDEX_FINGER_MCP_Y',
    'INDEX_FINGER_PIP_X', 'INDEX_FINGER_PIP_Y',
    'INDEX_FINGER_DIP_X', 'INDEX_FINGER_DIP_Y',
    'INDEX_FINGER_TIP_X', 'INDEX_FINGER_TIP_Y',
    'MIDDLE_FINGER_MCP_X', 'MIDDLE_FINGER_MCP_Y',
    'MIDDLE_FINGER_PIP_X', 'MIDDLE_FINGER_PIP_Y',
    'MIDDLE_FINGER_DIP_X', 'MIDDLE_FINGER_DIP_Y',
    'MIDDLE_FINGER_TIP_X', 'MIDDLE_FINGER_TIP_Y',
    'RING_FINGER_MCP_X', 'RING_FINGER_MCP_Y',
    'RING_FINGER_PIP_X', 'RING_FINGER_PIP_Y',
    'RING_FINGER_DIP_X', 'RING_FINGER_DIP_Y',
    'RING_FINGER_TIP_X', 'RING_FINGER_TIP_Y',
    'PINKY_MCP_X', 'PINKY_MCP_Y',
    'PINKY_PIP_X', 'PINKY_PIP_Y',
    'PINKY_DIP_X', 'PINKY_DIP_Y',
    'PINKY_TIP_X', 'PINKY_TIP_Y']

dataset_path = "./input"
train_imgs = glob.glob('./input/train/*')
test_imgs = glob.glob('./input/test/*')
save_dir_path = Path('./output')
df = extract_features_from_images(train_imgs[:100], feat_labels, save_dir_path)


skipped images count f0
